In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
mod = SentenceTransformer('all-mpnet-base-v2')

In [2]:
sentences = [
    'it caught him off guard that space smelled of seared steak',
    'she could not decide between painting her teeth or brushing her '
    'nails',
    "he thought there'd be sufficient time if he hid his watch",
    'the bees decided to have a mutiny against their queen',
    'the sign said there was road work ahead so she decided to speed '
    'up',
    "on a scale of one to ten what's your favorite flavor of color?",
    'flying stinging insects rebelled in opposition to the matriarch']

In [ ]:
embeddings = mod.encode(sentences)
embeddings.shape

In [ ]:
scores = cos_sim(embeddings[-1], embeddings[:-1])
scores

In [ ]:
sentences[scores.argmax().item()]